# XRP AMM price feed example

This is a tutorial notebook showing how to get a price feed for XRP Ledger's native AMM.

- In this example we get full historical price feeds for CRYPTO/XRP pair and XRP/USD pairs
- The notebook loads a lot of data on the first time, but subsequent runs are faster
- You cannot run this against public XRPL JSON-RPC nodes because they rate limit too much, get a node from QuikNode

See README for instructions on how to run.


## Setup 

Creating a JSON-RPC client.



In [1]:
from pathlib import Path
from xrpl.clients import JsonRpcClient
from xrpl_defi.ledger import get_latest_ledger_index
from xrpl_defi.utils.log import setup_console_logging

setup_console_logging()

# xrpl-py has a bad behavior of creating a hideen asyncio event loop,
# which does not work well in Jupyter notebooks..
# We use this hack to get around this behaviour.
import nest_asyncio
nest_asyncio.apply()

# Ask user to give the API key secret
settings_file = Path.home() / ".config" / "xrpl_defi" / "api-key.txt"
if not settings_file.exists():
    api_key = input("Enter your XRPL JSON-RPC node URL key: ")
    settings_file.parent.mkdir(parents=True, exist_ok=True)
    with open(settings_file, "w") as f:
        f.write(api_key)
else:
    api_key = settings_file.read_text().strip()

client = JsonRpcClient(api_key)

print(f"Connected to XRPL JSON-RPC client at {api_key[0:30]}..., latest ledger index: {get_latest_ledger_index(client):,}")

Connected to XRPL JSON-RPC client at https://still-few-isle.xrp-mai..., latest ledger index: 97,359,099


# Markets

- Choose markets to scan
- [Explore markets in XRPScan](https://xrpscan.com/amms)

In [2]:
AMM_MARKETS = [
    "rLjUKpwUVmz3vCTmFkXungxwzdoyrWRsFG",  # CRYPTO-XRP
    "rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3",  # RLUSD-XRP
]

## Fetch data

- Fetch data for all markets and store locally cached for the future runs

In [ ]:
from pathlib import Path
import pandas as pd
from xrpl_defi.markets.amm import fetch_amm_historical_payment_and_balances, prepare_amm_data

CACHE_PATH = Path.home() / ".cache" /  "xrpl-defi"
CACHE_PATH.mkdir(parents=True, exist_ok=True)
TRADES_CACHE_FILE = CACHE_PATH / "amm-data.parquet"

# Don't download full data (takes too long), just first trades
# MAX_LEDGER_INDEX = 87_544_747
MAX_LEDGER_INDEX = None  # To the tip of XRP ledger
MIN_LEDGER_INDEX = 95_000_000  # Ledger index to start fetching data from

if not TRADES_CACHE_FILE.exists():

    all_markets_df = None

    for market in AMM_MARKETS:
        print(f"Fetching trades for market {market}...")
        payment_tx_iter = fetch_amm_historical_payment_and_balances(
            client=client,
            account=market,
            min_ledger_index=MIN_LEDGER_INDEX,
            max_ledger_index=MAX_LEDGER_INDEX,
        )
        payment_txs = list(payment_tx_iter)
        print(f"Fetched {len(payment_txs)} txs for market {market}.")

        df = prepare_amm_data(payment_txs)

        if all_markets_df is None:
            all_markets_df = df 
        else:
            all_markets_df = pd.concat([all_markets_df, df], ignore_index=True)
        
    all_markets_df.to_parquet(TRADES_CACHE_FILE, compression="zstd")
    print(f"Trades data saved to {TRADES_CACHE_FILE}, size is {TRADES_CACHE_FILE.stat().st_size / 1024 / 1024:.2f} MB.")

else:
    # Use cached data
    print(f"Loading trades data from cache: {TRADES_CACHE_FILE}")
    all_markets_df = pd.read_parquet(TRADES_CACHE_FILE)


df = all_markets_df

Loading trades data from cache: /Users/moo/.cache/xrpl-defi/amm-data.parquet


# Explore data

- Show the first data entries

In [8]:
pd.set_option('display.float_format', '{:,.6f}'.format)

print(f"We have markets: {df['market'].unique()}")
df.head(5)

We have markets: ['rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3']


,timestamp,tx_hash,ledger_index,market,amm_asset_1,amm_asset_2,amm_asset_1_amount,amm_asset_2_amount,raw_json
0,2025-03-25 02:42:30+00:00,7887A71C647600BD6E032F7FA81D5CD6D39B24A71969C9...,95000091,rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3,XRP,RLUSD,"798,926.599093","1,945,593.736896","{""meta"": {""AffectedNodes"": [{""ModifiedNode"": {..."
1,2025-03-25 04:14:51+00:00,39EAD31C193014981BCB23E78E5D33CD5DB4DA99118D54...,95001515,rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3,XRP,RLUSD,"802,298.424632","1,937,332.512639","{""meta"": {""AffectedNodes"": [{""ModifiedNode"": {..."
2,2025-03-25 05:19:20+00:00,C8C0ABDD77E7DF27C8EB6CFF21F45884044FF9E22EC92B...,95002512,rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3,XRP,RLUSD,"800,863.826484","1,940,811.517088","{""meta"": {""AffectedNodes"": [{""ModifiedNode"": {..."
3,2025-03-25 06:19:31+00:00,05E0136E17B1D36F0354EC15C7564531312034EC57502B...,95003443,rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3,XRP,RLUSD,"799,652.684309","1,943,760.915040","{""meta"": {""AffectedNodes"": [{""ModifiedNode"": {..."
4,2025-03-25 07:22:52+00:00,EFC9E12E4AFE837298F30797C21D3423379B5B81DC16C0...,95004423,rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3,XRP,RLUSD,"801,393.816829","1,941,966.470159","{""meta"": {""AffectedNodes"": [{""ModifiedNode"": {..."


## Transform to OHLC

- Transform raw price events to OHLCV format
- Resample up to 1 day time frame


In [7]:
from xrpl_defi.markets.price import calculate_quote_price, calculate_ohlc

def process_market(pair_df, quote_token: str):
    """Process a single market's data and return OHLC"""
    price_df = calculate_quote_price(pair_df, quote_token)
    price_df = price_df.set_index("timestamp")
    price_series = price_df["quoted_price"]
    ohlc = calculate_ohlc(price_series, freq="1D")
    return ohlc

# Apply the function to each group and combine results
xrp_rlusd_df = process_market(df.loc[df.market == "rhWTXC2m2gGGA9WozUaoMm6kLAVPb1tcS3"], quote_token="RLUSD")
crypto_xrp_df = process_market(df.loc[df.market == "rLjUKpwUVmz3vCTmFkXungxwzdoyrWRsFG"], quote_token="XRP")

# Create a synthetic data for CYPTO/USD price
crypto_usd = crypto_xrp_df / xrp_rlusd_df["open"]

print("Example data")
xrp_rlusd_df.head(5)

AssertionError: DataFrame is empty

## XRP/RLUSD chart

- Draw the price chart

In [ ]:
from xrpl_defi.charts.price_chart import visualise_ohlc


fig = visualise_ohlc(xrp_rlusd_df, title="XRP / RLUSD")
fig.show()

## CRYPTO/XRP chart

- Draw the price chart

In [ ]:
fig = visualise_ohlc(crypto_xrp_df, title="XRP / RLUSD")
fig.show()

## CRYPTO/USD chart

- Draw CRYPTO/USD price chart, where we use XRP/RLUSD exchange rate as the rate for
  XRO quote token conversion

In [ ]:
fig = visualise_ohlc(crypto_xrp_df, title="CRYPTO / USD")
fig.show()